In [119]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import requests
import sqlite3
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import nbformat

In [78]:
# Lee el archivo CSV
df = pd.read_csv('portales_empleo.csv')

# Muestra las primeras filas del DataFrame
print(df.head())

   Unnamed: 0                                       Título Ubicación  \
0           0                                 Data Analyst    Madrid   
1           1  Business Intelligence Analyst (all genders)    Madrid   
2           2   Research Analyst (6 month Maternity Cover)    Madrid   
3           3    Senior Program Manager, Analyst Relations    Madrid   
4           4                       [RJ072] | Data Analyst    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...   
4  Join a forward-thinking company as a Data Anal...   

                                           Servicios  \
0  ['Tarjeta de restauranteSeguro de saludSeguro ...   
1  ['Flexible hybrid work environmentComprehensiv...   
2  ['Competitive benefitsCareer development oppor...   
3  ['E

In [79]:
df = df.drop(columns=['Unnamed: 0'])

In [80]:
# Eliminar filas duplicadas en su totalidad
df = df.drop_duplicates()

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 780 entries, 0 to 1035
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Título       780 non-null    object
 1   Ubicación    780 non-null    object
 2   Descripción  780 non-null    object
 3   Servicios    780 non-null    object
 4   Url          780 non-null    object
dtypes: object(5)
memory usage: 36.6+ KB


In [82]:
df.head(5)

,Título,Ubicación,Descripción,Servicios,Url
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1,Business Intelligence Analyst (all genders),Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...
2,Research Analyst (6 month Maternity Cover),Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...
4,[RJ072] | Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...


In [83]:
# Elimina el texto entre paréntesis en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\(.*?\)', '', regex=True)

# Elimina los signos '|' y '/'
df['Título'] = df['Título'].str.replace(r'[|/]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Elimina el texto entre corchetes en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\[.*?\]', '', regex=True)

# Elimina los signos '|', '/' y '-'
df['Título'] = df['Título'].str.replace(r'[|/-]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Muestra las primeras filas del DataFrame
df.head(5)

,Título,Ubicación,Descripción,Servicios,Url
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1,Business Intelligence Analyst,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...
2,Research Analyst,Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...
4,Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...


In [84]:
# Función para actualizar títulos
def actualizar_titulo(titulo):
    # Reemplazar 'jr' por 'Junior' y 'sr' por 'Senior'
    titulo = titulo.replace(' jr ', ' Junior ').replace(' sr ', ' Senior ')
    titulo = titulo.replace('jr ', 'Junior ').replace('sr ', 'Senior ')
    titulo = titulo.replace(' jr', ' Junior').replace(' sr', ' Senior')
    return titulo

# Aplicar la función de actualización a la columna 'Título'
df['Título'] = df['Título'].apply(actualizar_titulo)

In [85]:
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)

In [86]:
import re

In [87]:
# Listas de palabras clave (nombres de empresas y sectores)
empresas = ['santander', 'amazon', 'microsoft', 'vie', 'bcg', 'mckinsey']  # Añadir más nombres de empresas
sectores = ['marketing', 'medtech', 'fintech', 'ecommerce', 'finance', 'game','genai','digital', 'tax', 'governance', 'energy' ]  # Añadir más sectores

# Convertir las listas a un conjunto de expresiones regulares para facilitar la búsqueda
regex_empresas = re.compile(r'\b(' + '|'.join(empresas) + r')\b', re.IGNORECASE)
regex_sectores = re.compile(r'\b(' + '|'.join(sectores) + r')\b', re.IGNORECASE)

# Función para extraer empresa
def extraer_empresa(titulo):
    match = regex_empresas.search(titulo)
    return match.group(0) if match else None

# Función para extraer sector
def extraer_sector(titulo):
    match = regex_sectores.search(titulo)
    return match.group(0) if match else None

# Función para limpiar el título
def limpiar_titulo(titulo):
    # Eliminar nombres de empresas y sectores del título
    titulo = regex_empresas.sub('', titulo)
    titulo = regex_sectores.sub('', titulo)
    # Eliminar espacios extra
    return ' '.join(titulo.split())

# Aplicar las funciones a las columnas del DataFrame
df['Empresa'] = df['Título'].apply(extraer_empresa)
df['Sector'] = df['Título'].apply(extraer_sector)
df['Título'] = df['Título'].apply(limpiar_titulo)

In [88]:
df.head(12)

,Título,Ubicación,Descripción,Servicios,Url,Expertise,Empresa,Sector
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...,Regular,None,None
1,Business Intelligence Analyst,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...,Regular,None,None
2,Research Analyst,Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...,Regular,None,None
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...,Senior,None,None
4,Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...,Regular,None,None
5,Data Analyst Salesforce CRM Analytics,Madrid,Sin descripción,[],https://jobleads.com/job/ec3395299d0f0d3a84dfa...,Regular,None,None
6,"Business Analyst, PV Global Ops",Madrid,Join a dynamic team at the forefront of entert...,[' Excel (including VBA) Data visualization to...,https://jobleads.com/job/e860353a7964741360fae...,Regular,None,None
7,Global Markets Sales and Trading OffCycle 2025...,Madrid,Sin descripción,[],https://jobleads.com/job/e4c754a957f82738907f4...,Regular,None,None
8,Desk Quant Analyst,Madrid,An innovative global investment management fir...,[' Programming proficiency Attention to detail...,https://jobleads.com/job/e7b34be8695587f9c91d7...,Regular,None,None
9,Business Analyst,Madrid,Join a forward-thinking firm where you will co...,['Mentorship ProgramsProfessional Development ...,https://jobleads.com/job/e2ad695c320220edb4118...,Regular,None,None


In [89]:
# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'inglés' if 'inglés' in x else None)

# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'English' if 'English' in x else None)

# Detecta las palabras clave "Híbrido", "Remoto" o "Presencial" y asigna a la nueva columna 'Modalidad'
df['Modalidad'] = df['Título'].apply(lambda x: 'Híbrido' if 'Híbrido' in x else ('Remoto' if 'Remoto' in x else ('Presencial' if 'Presencial' in x else None)))

df['Otro Idioma'] = df['Título'].apply(lambda x: 'english' if 'english' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'french' if 'french' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'francés' if 'francés' in x else None)

In [90]:
# Función para detectar los idiomas
def detectar_idiomas(titulo):
    if 'inglés' in titulo.lower() or 'english' in titulo.lower():
        return 'Inglés'
    elif 'francés' in titulo.lower() or 'french' in titulo.lower():
        return 'Francés'
    # Puedes añadir más idiomas aquí
    return None

# Aplicar la función para detectar idiomas
df['Otro Idioma'] = df['Título'].apply(detectar_idiomas)

In [91]:
titulos_agrupados = df['Otro Idioma'].value_counts()
print(titulos_agrupados)

Otro Idioma
Inglés     10
Francés     4
Name: count, dtype: int64


In [92]:
# Diccionario de ubicaciones y sus modificaciones
ubicaciones_dict = {
    'Boadilla del Monte': 'Madrid',
    'Alcobendas': 'Madrid',
    'Leganés': 'Madrid',
    'Getafe': 'Madrid',
    'Barcelona, Centro': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Pozuelo de Alarcón': 'Madrid',
    'Tres Cantos': 'Madrid',
    'Villaviciosa de Odón': 'Madrid',
    'Sant Just Desvern': 'Cataluña'
}

# Función para reemplazar la ubicación según el diccionario
df['Ubicación'] = df['Ubicación'].replace(ubicaciones_dict)

In [93]:
titulos_agrupados = df['Ubicación'].value_counts()
print(titulos_agrupados)

Ubicación
Cataluña     421
Madrid       355
Andalucía      4
Name: count, dtype: int64


In [94]:
titulos_agrupados = df['Empresa'].value_counts()
print(titulos_agrupados)

Empresa
Santander    9
Amazon       2
VIE          2
BCG          1
Mckinsey     1
Microsoft    1
Name: count, dtype: int64


In [95]:
# Remover caracteres innecesarios y estandarizar
df['Título'] = df['Título'].str.replace(r'[^\w\s]', '', regex=True).str.strip().str.lower()

In [96]:
# Crear columna de 'Puesto' principal basado en palabras clave
def clasificar_rol(titulo):
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    else:
        return 'Other'

df['Puesto'] = df['Título'].apply(clasificar_rol)

In [97]:
# Reordenar columnas en el orden especificado
columnas_ordenadas = [
    'Título', 'Puesto', 'Expertise', 'Empresa', 'Modalidad', 'Sector', 
    'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma'
]
df = df[columnas_ordenadas]

In [98]:
# Función para clasificar el puesto basado en palabras clave extendidas
def clasificar_rol(titulo):
    titulo = titulo.lower()
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'data manager' in titulo or 'manager data' in titulo:
        return 'Data Manager'
    elif 'head of data' in titulo:
        return 'Data Manager'
    elif 'data expert' in titulo:
        return 'Data Expert'
    elif 'big data architect' in titulo:
        return 'Big Data Architect'
    elif 'manager data cloud' in titulo:
        return 'Data Manager Cloud'
    elif 'data analytics' in titulo:
        return 'Data Analytics'
    elif 'ai innovation architect' in titulo:
        return 'AI Innovation Architect'
    elif 'data management engineer' in titulo:
        return 'Data Manager'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    elif 'data developer' in titulo:
        return 'Data Developer'
    elif 'project manager' in titulo:
        return 'Project Manager'
    else:
        return 'Other'

# Aplicar la función a la columna "Título" para crear o actualizar la columna "Puesto"
df['Puesto'] = df['Título'].apply(clasificar_rol)

# Mostrar el DataFrame resultante
print(df)

                                            Título                     Puesto  \
0                                     data analyst               Data Analyst   
1                    business intelligence analyst      Business Intelligence   
2                                 research analyst               Data Analyst   
3         senior program manager analyst relations               Data Analyst   
4                                     data analyst               Data Analyst   
...                                            ...                        ...   
1017              senior machine learning engineer  Machine Learning Engineer   
1018                      data engineering manager              Data Engineer   
1019                      data engineering manager              Data Engineer   
1034  it stibo step inhouse product data developer             Data Developer   
1035  it stibo step inhouse product data developer             Data Developer   

     Expertise Empresa Moda

In [99]:
# Función para limpiar nombres de ciudades y códigos de puesto directamente en "Título"
def limpiar_titulo(titulo):
    # Eliminación de nombres de ciudades comunes en títulos
    ciudades = ["barcelone", "alicante", "catalonia", "madrid", "bilbao"]  # Añadir otras ciudades según sea necesario
    for ciudad in ciudades:
        titulo = re.sub(rf"\b{ciudad}\b", "", titulo, flags=re.IGNORECASE)
    
    # Eliminación de códigos de puesto que suelen ser letras y números juntos
    titulo = re.sub(r"\b[a-zA-Z]{2,3}\d{3}\b", "", titulo)
    
    # Eliminación de espacios extras generados por las sustituciones
    titulo = re.sub(r"\s+", " ", titulo).strip()
    return titulo

# Aplicar la función de limpieza a la columna "Título" para modificarla directamente
df['Título'] = df['Título'].apply(limpiar_titulo)

# Función para extraer herramientas tecnológicas
def extraer_herramientas(titulo):
    herramientas = ["Salesforce", "CRM", "Power BI", "Qlik", "Python", "Tableau", "SAS", "SAP", "BigQuery", "Datastage", "Analytics", "Pentaho", "Spark"]  # Añadir otras herramientas según sea necesario
    herramientas_encontradas = [herramienta for herramienta in herramientas if herramienta.lower() in titulo.lower()]
    return ", ".join(herramientas_encontradas) if herramientas_encontradas else None

# Crear la columna "Herramientas" con las herramientas tecnológicas encontradas en el título
df['EntornoTEC'] = df['Título'].apply(extraer_herramientas)

In [100]:
def convertir_abreviaciones(titulo):
    titulo = titulo.lower()  # Convertir a minúsculas para comparación
    if 'sr' in titulo:
        titulo = titulo.replace('sr', 'senior')
    if 'jr' in titulo:
        titulo = titulo.replace('jr', 'junior')
    return titulo

# Aplicar la función para convertir los títulos
df['Título'] = df['Título'].apply(convertir_abreviaciones)

In [101]:
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title or 'associate' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title or 'becarioa' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title or 'head' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)

In [102]:
titulos_agrupados = df['Puesto'].value_counts()
print(titulos_agrupados.to_string())

Puesto
Data Analyst                 353
Data Engineer                161
Data Scientist               126
Other                         71
Data Specialist               19
Machine Learning Engineer     19
Business Intelligence         11
Consultant                     6
Data Analytics                 6
Data Developer                 4
Project Manager                2
Data Expert                    1
AI Innovation Architect        1


In [103]:
# Filtrar filas que contengan el texto deseado en la columna "Título"
filtro = df[df['Puesto'].str.contains("Other", case=False, na=False)]

# Mostrar las filas filtradas
print(filtro)

                                                Título Puesto Expertise  \
12   sales trading and structuring graduate program...  Other   Regular   
76       salesforce implementation lead crm automation  Other      Lead   
77       salesforce implementation lead crm automation  Other      Lead   
87                coordinadora control de gestión d287  Other   Regular   
92                           jefea de proyecto hibrido  Other   Regular   
..                                                 ...    ...       ...   
965                                   data bi engineer  Other   Regular   
969                            data migration engineer  Other   Regular   
970                          data integration engineer  Other   Regular   
982                 senior data platform engineer link  Other    Senior   
983                      senior data platform engineer  Other    Senior   

    Empresa Modalidad     Sector Ubicación  \
12     None      None       None    Madrid   
76     

In [104]:
# Función para agregar espacios entre palabras correctamente
def add_spaces(text):
    # Añade un espacio entre palabras que estén pegadas (ej: "SnacksWeekly" -> "Snacks Weekly")
    corrected_text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)
    return corrected_text

# Aplicar la función a la columna "Servicios"
df['Servicios'] = df['Servicios'].apply(add_spaces)

In [105]:
# Lista de palabras comunes a eliminar
common_words = r'\b(de|a|in|or|and|the|of|en|con|para|por|y|del|la|los|las|un|una)\b'

# Definición de las palabras clave para cada categoría
benefits_keywords = r'\b(Tarjeta de restaurante|Teletrabajo opcional|Flexible hours|Modern office environment|Continuous learning programs|Mentoring programs|Collaborative culture|Oportunidades de crecimiento profesional|Paquete competitivo de compensación y beneficios|Retirement program|Well-being programs|Diversity initiatives|Work-life balance|Remote work)\b'
skills_keywords = r'\b(Neural networks|Bayesian modeling|Data storytelling|Análisis de datos|Métodos estadísticos|Business Intelligence|Data Science|Machine Learning|Deep Learning|Data Visualization|Problem Solving|Data Analysis|Reporting|Data governance|Statistical Analysis|Predictive Modeling)\b'
tools_keywords = r'\b(Python|R|Pyspark|Power BI|Tableau|Databricks|GCP|SAS|SQL|Excel|VBA|Google Analytics|Adobe Analytics|Domo|Snowflake|TensorFlow|Keras|Scikit-learn|Power Platforms|Jupyter Notebooks|Azure|AWS|Docker|GitHub|Streamlit|Pytorch|BigQuery|SQL)\b'
education_keywords = r'\b(Degree in Computing or Statistics|Bachelor\'s Degree|Master\'s Degree|PhD|Licenciatura|Grado|Engineering Degree|Data Science Degree|Statistics Degree|Computer Science Degree|Mathematics Degree)\b'

# Función para limpiar texto de palabras comunes
def remove_common_words(text):
    return re.sub(common_words, '', text, flags=re.IGNORECASE)

# Función para clasificar las palabras en las categorías
def classify_services(services):
    # Limpiar palabras comunes
    services_cleaned = remove_common_words(services)
    
    # Clasificar términos en categorías
    benefits = re.findall(benefits_keywords, services_cleaned, flags=re.IGNORECASE)
    skills = re.findall(skills_keywords, services_cleaned, flags=re.IGNORECASE)
    tools = re.findall(tools_keywords, services_cleaned, flags=re.IGNORECASE)
    education = re.findall(education_keywords, services_cleaned, flags=re.IGNORECASE)
    
    # Devolver resultados en columnas separadas
    return pd.Series({
        'Beneficios': ', '.join(benefits) if benefits else None,
        'Habilidades': ', '.join(skills) if skills else None,
        'Herramientas': ', '.join(tools) if tools else None,
        'Educación': ', '.join(education) if education else None,
    })

# Aplicar la función a cada fila
df_new = df['Servicios'].apply(classify_services)

# Combinar los resultados en un nuevo DataFrame
result_df = pd.concat([df, df_new], axis=1)

# Mostrar el DataFrame resultante
print(result_df)

                                            Título                     Puesto  \
0                                     data analyst               Data Analyst   
1                    business intelligence analyst      Business Intelligence   
2                                 research analyst               Data Analyst   
3         senior program manager analyst relations               Data Analyst   
4                                     data analyst               Data Analyst   
...                                            ...                        ...   
1017              senior machine learning engineer  Machine Learning Engineer   
1018                      data engineering manager              Data Engineer   
1019                      data engineering manager              Data Engineer   
1034  it stibo step inhouse product data developer             Data Developer   
1035  it stibo step inhouse product data developer             Data Developer   

     Expertise Empresa Moda

In [106]:
result_df.head(15)

,Título,Puesto,Expertise,Empresa,Modalidad,Sector,Ubicación,Descripción,Servicios,Otro Idioma,EntornoTEC,Beneficios,Habilidades,Herramientas,Educación
0,data analyst,Data Analyst,Regular,None,None,None,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restaurante Seguro de salud Segur...,None,None,None,None,"Excel, R, Python","Licenciatura, Licenciatura, Licenciatura, Lice..."
1,business intelligence analyst,Business Intelligence,Regular,None,None,None,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environment Comprehensi...,None,None,None,"Data Governance, Data Science",Tableau,Bachelor's Degree
2,research analyst,Data Analyst,Regular,None,None,None,Madrid,An established industry player is seeking a Re...,['Competitive benefits Career development oppo...,None,None,None,"Data Analysis, Data Visualization","Excel, Tableau, Excel, Tableau",None
3,senior program manager analyst relations,Data Analyst,Senior,None,None,None,Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,None,None,None,None,None,Bachelor's Degree
4,data analyst,Data Analyst,Regular,None,None,None,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hours Health insurance Life...,None,None,None,"Data Analysis, Data Visualization","Excel, R, Python","Bachelor's degree, Bachelor's degree, Bachelor..."
5,data analyst salesforce crm analytics,Data Analyst,Regular,None,None,None,Madrid,Sin descripción,[],None,"Salesforce, CRM, Analytics",None,None,None,None
6,business analyst pv global ops,Data Analyst,Regular,None,None,None,Madrid,Join a dynamic team at the forefront of entert...,[' Excel (including V B A) Data visualization ...,None,None,None,"Data visualization, Data analysis","Excel, Tableau, Tableau","Bachelor's degree, Master's degree"
7,global markets sales and trading offcycle 2025...,Data Analyst,Regular,None,None,None,Madrid,Sin descripción,[],None,None,None,None,None,None
8,desk quant analyst,Data Analyst,Regular,None,None,None,Madrid,An innovative global investment management fir...,[' Programming proficiency Attention to detail...,None,None,None,None,Python,"Engineering Degree, Computer Science Degree"
9,business analyst,Data Analyst,Regular,None,None,None,Madrid,Join a forward-thinking firm where you will co...,['Mentorship Programs Professional Development...,None,None,None,"Problem Solving, Data Analysis",None,"Bachelor's Degree, Master's Degree"


In [107]:
result_df.to_csv("finalResult.csv", index = False)

In [127]:

# Cargar los datos desde un archivo CSV
data = pd.read_csv('finalResult.csv')

# Verificar las primeras filas de los datos
print(data.head())

# Preparar los datos para el clustering (selecciona las características relevantes)
X = data[['Puesto', 'Expertise', 'Sector']]

# Convertir variables categóricas a numéricas
X['Puesto'] = X['Puesto'].astype('category').cat.codes
X['Expertise'] = X['Expertise'].astype('category').cat.codes
X['Sector'] = X['Sector'].astype('category').cat.codes

# Aplicar K-means para segmentar los datos
kmeans = KMeans(n_clusters=3, random_state=0)
data['Segmento'] = kmeans.fit_predict(X)

# Visualizar la cantidad de ofertas por segmento
segment_counts = data['Segmento'].value_counts().reset_index()
segment_counts.columns = ['Segmento', 'Cantidad']

# Gráfico de barras mostrando la cantidad de ofertas por segmento
fig_barras = go.Figure(data=[
    go.Bar(x=segment_counts['Segmento'], y=segment_counts['Cantidad'], 
           marker=dict(color='royalblue'))
])

# Configurar el layout del gráfico de barras
fig_barras.update_layout(
    title='Cantidad de Ofertas por Segmento',
    xaxis_title='Segmento',
    yaxis_title='Cantidad de Ofertas'
)

# Guardar el gráfico como archivo HTML y abrirlo en el navegador
pio.write_html(fig_barras, 'grafico_barras.html')
import webbrowser
webbrowser.open('grafico_barras.html')

# Gráfico de dispersión mostrando los diferentes segmentos
fig_dispersion = go.Figure()

# Agregar trazas para cada segmento
for segmento in data['Segmento'].unique():
    seg_data = data[data['Segmento'] == segmento]
    fig_dispersion.add_trace(go.Scatter(
        x=seg_data['Puesto'], 
        y=seg_data['Expertise'], 
        mode='markers',
        name=f'Segmento {segmento}',
        text=seg_data['Descripción'],  # Mostrar descripción al pasar el ratón
        marker=dict(size=10)
    ))

# Configurar el layout del gráfico de dispersión
fig_dispersion.update_layout(
    title='Resultados del Análisis de Segmentación',
    xaxis_title='Puesto',
    yaxis_title='Expertise',
    legend_title='Segmentos',
    hovermode='closest'
)

# Guardar el gráfico de dispersión como archivo HTML y abrirlo en el navegador
pio.write_html(fig_dispersion, 'grafico_dispersion.html')
webbrowser.open('grafico_dispersion.html')


                                     Título                 Puesto Expertise  \
0                              data analyst           Data Analyst   Regular   
1             business intelligence analyst  Business Intelligence   Regular   
2                          research analyst           Data Analyst   Regular   
3  senior program manager analyst relations           Data Analyst    Senior   
4                              data analyst           Data Analyst   Regular   

  Empresa Modalidad Sector Ubicación  \
0     NaN       NaN    NaN    Madrid   
1     NaN       NaN    NaN    Madrid   
2     NaN       NaN    NaN    Madrid   
3     NaN       NaN    NaN    Madrid   
4     NaN       NaN    NaN    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...  

C:\Users\guerc\AppData\Local\Temp\ipykernel_6188\2566328237.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\guerc\AppData\Local\Temp\ipykernel_6188\2566328237.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\guerc\AppData\Local\Temp\ipykernel_6188\2566328237.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

True

In [112]:


# Asegurarte de que el DataFrame tiene los nombres de columnas correctos
result_df.columns = [
    "Título", "Puesto", "Expertise", "Empresa", "Modalidad", "Sector", 
    "Ubicación", "Descripción", "Servicios", "Otro_Idioma", "EntornoTEC", 
    "Beneficios", "Habilidades", "Herramientas", "Educación"
]

# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Eliminar la tabla si existe, y luego crearla de nuevo con los nombres de columnas correctos
cursor.execute("DROP TABLE IF EXISTS job_data")
cursor.execute('''
    CREATE TABLE job_data (
        Título TEXT,
        Puesto TEXT,
        Expertise TEXT,
        Empresa TEXT,
        Modalidad TEXT,
        Sector TEXT,
        Ubicación TEXT,
        Descripción TEXT,
        Servicios TEXT,
        Otro_Idioma TEXT,
        EntornoTEC TEXT,
        Beneficios TEXT,
        Habilidades TEXT,
        Herramientas TEXT,
        Educación TEXT
    )
''')

# Insertar el DataFrame en la tabla de SQLite
result_df.to_sql('job_data', conn, if_exists='append', index=False)

# Verificar que los datos se hayan guardado correctamente
df_check = pd.read_sql("SELECT * FROM job_data LIMIT 5", conn)
print(df_check)

# Cerrar la conexión
conn.close()


                                     Título                 Puesto Expertise  \
0                              data analyst           Data Analyst   Regular   
1             business intelligence analyst  Business Intelligence   Regular   
2                          research analyst           Data Analyst   Regular   
3  senior program manager analyst relations           Data Analyst    Senior   
4                              data analyst           Data Analyst   Regular   

  Empresa Modalidad Sector Ubicación  \
0    None      None   None    Madrid   
1    None      None   None    Madrid   
2    None      None   None    Madrid   
3    None      None   None    Madrid   
4    None      None   None    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...  

In [113]:


# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Leer los datos de la tabla 'job_data' en un DataFrame de pandas
query = "SELECT * FROM job_data LIMIT 5"
df_check = pd.read_sql(query, conn)

# Mostrar el DataFrame para verificar que los datos se hayan guardado
print(df_check)

# Cerrar la conexión
conn.close()


                                     Título                 Puesto Expertise  \
0                              data analyst           Data Analyst   Regular   
1             business intelligence analyst  Business Intelligence   Regular   
2                          research analyst           Data Analyst   Regular   
3  senior program manager analyst relations           Data Analyst    Senior   
4                              data analyst           Data Analyst   Regular   

  Empresa Modalidad Sector Ubicación  \
0    None      None   None    Madrid   
1    None      None   None    Madrid   
2    None      None   None    Madrid   
3    None      None   None    Madrid   
4    None      None   None    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...  

In [114]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Consultar los datos donde la ubicación es "Madrid"
query = "SELECT * FROM job_data WHERE Ubicación = 'Madrid'"
df_madrid = pd.read_sql(query, conn)

# Mostrar el DataFrame con los registros de Madrid
print(df_madrid)

# Cerrar la conexión
conn.close()

                                       Título                 Puesto  \
0                                data analyst           Data Analyst   
1               business intelligence analyst  Business Intelligence   
2                            research analyst           Data Analyst   
3    senior program manager analyst relations           Data Analyst   
4                                data analyst           Data Analyst   
..                                        ...                    ...   
350                             data engineer          Data Engineer   
351                             data engineer          Data Engineer   
352                             data engineer          Data Engineer   
353               data analyst 100 presencial           Data Analyst   
354                       senior data analyst           Data Analyst   

    Expertise Empresa   Modalidad Sector Ubicación  \
0     Regular    None        None   None    Madrid   
1     Regular    None      